In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
import threading
import time
import random
import os
import concurrent.futures

os.environ['GH_TOKEN'] = 'MARC LO TIENE'

asins = set()
lock = threading.Lock()

def scrape_termino(termino):
    options = webdriver.FirefoxOptions()
    options.add_argument("--headless")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.set_preference("dom.webdriver.enabled", False)
    options.set_preference("useAutomationExtension", False)
    user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.59",
    "Mozilla/5.0 (Linux; Android 10; SM-A505FN) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.120 Mobile Safari/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Mobile/15E148 Safari/604.1",
    ]
    options.add_argument(f"user-agent={random.choice(user_agents)}")
    driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()), options=options)
    try:
        base_url = f"https://www.amazon.es/s?k={termino.replace(' ', '+')}&page="
        page = 1
        while page <= 7:
            url = f"{base_url}{page}"
            print(f"Scrapeando término '{termino}', página {page}: {url}")
            max_reintentos = 3
            reintentos = 0
            while reintentos < max_reintentos:
                try:
                    driver.get(url)
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-asin]')))
                    productos = driver.find_elements(By.CSS_SELECTOR, 'div[data-asin]')
                    data_asins = set()
                    for p in productos:
                        try:
                            asin = p.get_attribute("data-asin")
                            if asin:
                                data_asins.add(asin)
                        except Exception as e:
                            print(f"Error al obtener 'data-asin': {e}")
                            continue
                    with lock:
                        tamaño_anterior = len(asins)
                        asins.update(data_asins)
                        nuevos_asins = len(asins) - tamaño_anterior
                    print(f"Recogidos {len(data_asins)} ASINs en la página {page}, de los cuales {nuevos_asins} son nuevos.")
                    break
                except Exception as e:
                    reintentos += 1
                    print(f"Error en la página {page} del término '{termino}': {e}. Reintento {reintentos}/{max_reintentos}")
                    if reintentos == max_reintentos:
                        print(f"Se agotaron los reintentos para la página {page} del término '{termino}'.")
                        continue
            time.sleep(random.uniform(1, 3))
            page += 1
    finally:
        driver.quit()
        
terminos_busqueda = ["moviles", "phones", "moviles baratos", "movil iphones", "movil xiaomi", "movil samsung", "moviles españa", "moviles top", "moviles buenos"]

with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    executor.map(scrape_termino, terminos_busqueda)

with open("asins.txt", "w") as f:
    f.write("\n".join(asins))
    
print(f"Scraping completado. Total ASINs únicos guardados: {len(asins)}")

Scraping completado. Total ASINs únicos guardados: 0


# Codi per carregar totes les pàgines 

In [ ]:
import requests
import concurrent.futures

API_KEYS = [
    "000f8774b5f8782161820d3a038fddc4"
]
current_api_index = 0 

with open("asins.txt", "r") as file:
    ASIN_LIST = [line.strip() for line in file.readlines()]

products_data = []

def fetch_product_data(asin):
    local_current_api_index = current_api_index  # Variable local para cada hilo
    for _ in range(len(API_KEYS)):
        payload = {
            'api_key': API_KEYS[local_current_api_index],
            'url': f'https://www.amazon.es/dp/{asin}',
            'output_format': 'json',
            'autoparse': 'true',
            'retry_404': 'true'
        }
        response = requests.get('https://api.scraperapi.com/', params=payload)
        if response.status_code == 200:
            product_info = response.json()
            return product_info
        else:
            print(f"Error con API_KEY {local_current_api_index + 1}: {API_KEYS[local_current_api_index]}")
            local_current_api_index = (local_current_api_index + 1) % len(API_KEYS)
            print(f"Usando API_KEY {local_current_api_index + 1}: {API_KEYS[local_current_api_index]}")
    return None

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    results = executor.map(fetch_product_data, ASIN_LIST)

for product_info in results:
    if product_info is not None:
        products_data.append(product_info)

print(products_data)

[{'name': 'realme Smartphone 12x 5G 8+256 GB, Smartphone de 6,72 Pulgadas, Cámara AI de 50 MP, Dual Standby 5G, Pantalla de 120 Hz, Batería de 5.000 mAh, Carga SUPERVOOC de 45W, Verde (sin Adaptador)', 'product_information': {'Marca': 'realme', 'Fabricante': 'realme', 'Modelo': 'RMX3997', 'Nombre del modelo': '12X', 'Año del modelo': '2020', 'Dimensiones del paquete': '16,6 x 7,6 x 0,8 cm; 200 g', 'Pilas': '1 Litio Ion necesaria(s), incluida(s)', 'Número de modelo del producto': 'RMX3997', 'Número de producto': '12x', 'Capacidad de la memoria': '256 GB', 'Capacidad de la memoria flash instalada': '8 GB', 'Capacidad de la memoria RAM instalada': '8 GB', 'Capacidad de almacenamiento digital': '8 GB', 'Sistema operativo': 'Android', 'Velocidad del procesador': '2,6 GHz', 'Componentes incluidos': 'Cable USB', 'Número de productos': '1', 'Tamaño de la pantalla': '6,72 Pulgadas', 'Tipo de pantalla': 'LCD', 'Relación de aspecto': '16:9', 'Pilas / baterías incluidas': 'Sí', 'Pilas / baterías n

In [11]:
import csv
import re

def find_value_by_keywords(data_dict, keywords, pattern=None, default="null"):
    for key, value in data_dict.items():
        if any(keyword.lower() in key.lower() for keyword in keywords):
            if pattern is None or re.search(pattern, str(value)):
                return value
    return default
def safe_get(value, default="null"):
    return str(value).replace("€", "").strip() if value else default
with open('moviles.csv', mode='w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerow(["URL", "Asin", "Precio", "Precio Inicial", "Título", "Estrellas", "Opiniones", "Marca", "Modelo", "Año del modelo", "Dimensiones",
                     "RAM", "Memoria", "Sistema operativo", "Resolución pantalla", "Tamaño pantalla", "Relación aspecto", "Peso", "Tecnología conectividad", "Batería"])

    for product in products_data:
        product_information = product.get("product_information", {})
        
        url = safe_get(product.get("brand_url"))
        asin = safe_get(product_information.get("ASIN"))
        product_price = safe_get(product.get("pricing"))
        product_original_price = safe_get(product.get("list_price", product_price))
        
        product_title = safe_get(product.get("name"))
        product_star_rating = safe_get(product.get("average_rating"))
        product_num_ratings = safe_get(product.get("total_reviews"))

        marca = find_value_by_keywords(product_information, ["fabricante", "marca"])
        modelo = find_value_by_keywords(product_information, ["modelo"])
        ano = find_value_by_keywords(product_information, ["año"], r"\b\d{4}\b")
        dimensiones = find_value_by_keywords(product_information, ["dimensiones", "tamaño", "medidas"], r"\d+")
        ram = find_value_by_keywords(product_information, ["RAM", "memoria RAM"], r"\b(1|2|4|8|16|32|64)\s?GB\b")
        memoria = find_value_by_keywords(product_information, ["ROM", "memoria", "capacidad"], r"\b(32|64|128|256|512|1024)\s?(GB|TB)\b")
        sistema_operativo = safe_get(product_information.get("Sistema operativo"))
        resolucion_pantalla = find_value_by_keywords(product_information, ["resolucion", "resolución"], r"\d+")
        tamano_pantalla = find_value_by_keywords(product_information, ["pantalla", "pulgadas"], r"\d+")
        relacion_aspecto = find_value_by_keywords(product_information, ["relacion", "aspecto"], r"\d+:\d+")
        peso = find_value_by_keywords(product_information, ["peso", "masa"], r"\b\d+\s?g\b")
        tecnologia_conectividad = find_value_by_keywords(product_information, ["tecnología", "conectividad"], r"\b(2G|3G|4G|5G)\b")
        bateria = find_value_by_keywords(product_information, ["batería", "capacidad batería"], r"\b\d+\s?mAh\b")

        writer.writerow([
            url, asin, product_price, product_original_price, product_title, product_star_rating, product_num_ratings,
            marca, modelo, ano, dimensiones, ram, memoria, sistema_operativo, resolucion_pantalla, tamano_pantalla, relacion_aspecto, peso, tecnologia_conectividad, bateria
        ])